You can use this notebook to visulalize the quantum correlators. You need to perform a semiclassical simulation (with or without backreaction) and store "data", and "correlators". It assumes that both data-types are saved at the same timestpes. If not, you need to modify appropriately.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpecialFunctions
using FunctionZeros
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_false_mPV100.0_dk_denom_15_kmax10.0_lmax30.0"
your_dir = dir*par

In [ ]:
# set the number of grid points you used
D = 3
Nr = 128*2^D + 3

# make directory to save figures
out_dir = "./correlators_tr/"*par*"/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");
(its_crlt, all_filenames_crlt) = list_h5_files(your_dir*"/data_$(Nr)", prefix="correlators_");

# classical data
println("length(its)=",length(its))
println("its",its)

# correlators
println("length(its_crlt)=",length(its_crlt))
println("its_crlt=",its_crlt)

Define a function to plot the real part of equal time correlators. The structure of the correlators is given before the function. You can modify it to plot the imaginary part

In [ ]:
"""
for the correlators the format (r, r)
    it is the 2pt function for the fields:
    1) ∂t_utld_kl = r^l*α*πq/(B*A^0.5)
    2) ∂r_utld_kl = l*r^(l-1)*uq + r^l*ψq
    3) utld_kl = uq*r^l
    4) πq
    5) ψq
"""

function plot_crlt(it, crlt_i, ri_min, ri_max)

    # if you have not saved data and correlators on the same timesteps
    # you need to change it below, accordingly
    it_data = its[it]
    it_str_data  = lpad(it_data, 4, "0")
    
    it_crlt = its_crlt[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    
    v = h5read(your_dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    
    crlt = h5read(your_dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    
    # if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
    t_data   =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    t_crlt   =  h5readattr(your_dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5", "/")["time"]
    
    # checks if time instant is the same for all types of data
    @assert t_data ≈ t_crlt

    # change "real" -> "imag" below, if you want the imaginary part of the correlator
    println("max(abs(real(crlt[$(crlt_i)]))) = ",maximum(abs.(real(crlt[ri_min:ri_max, ri_min:ri_max, crlt_i]))))
    contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        # change "real" -> "imag" below, if you want the imaginary part of the correlator
        real(crlt[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r1", yaxis="r2",
        #title = L"\mathcal{R}\left[ C(r_1,r_2)\right]"*" @ t=$(t_crlt)",)
        title = "real(crlt[$(crlt_i)]) @t=$(t_crlt)",)
    AH = find_AH(r, B, A, KB)
    plot!([AH], seriestype="vline", label="AH", linewidth=3, color = "black")
    plot!([AH], seriestype="hline", label="AH", linewidth=3, color = "black")
end

In [ ]:
# index of min and max, in r domain
ri_min = 3
ri_max = 550 #Nr
# timestep (counts within its_crlt list
ti = 100
# which correlator you want (see previous cell for the structure). There are 5 different (1-5).
crlt_i = 3

plt = plot_crlt(ti, crlt_i, ri_min, ri_max)
# uncomment below if you want to save fig
#savefig(plt, out_dir*"/crlt_$(crlt_i)-$(ti).pdf")

The loop below plots the chosen correlator for all the saved timesteps, and saves if as pdf (you can also choose png).

In [ ]:
#crlt_i = 3
#for i ∈ 1:length(its_crlt)
#    plt = plot_crlt(i, crlt_i, ri_min, ri_max)
#    savefig(plt, out_dir*"/crlt_$(crlt_i)-$(i).pdf")
#end

You can also plot the scalar field in time with the function below:

In [ ]:
function plot_Φ(it, ri_min, ri_max)

    it_data = its[it]
    it_str_data  = lpad(it_data, 4, "0")
    
    v = h5read(your_dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
    Φ  = v[:,1]
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
        
    # if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
    t_data   =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    
    plot(r[ri_min:ri_max], Φ[ri_min:ri_max], title = "t=$(t_data)",
        label = "Φ(r)", linewidth = 2, frame = true, wsize = (800,400),
    xlim = (0,15), xlabel = "r")
    
    AH = find_AH(r, B, A, KB)
    plot!([AH], seriestype="vline", label="AH", linewidth=2, color = "black", legendfontsize=13)
end

In [ ]:
plot_Φ(100, 3, 1027)

In [ ]:
# uncomment if you want to save Phi(r) for each timestep (or tune which timsteps you want)

#for i ∈ 1:length(its_crlt)
#    plt = plot_Φ(i, 3, 1027)
#    savefig(plt, out_dir*"/Phi-$(i).pdf")
#end